In [1]:
from langchain.llms.base import LLM
from typing import List, Optional

import requests

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import re
import time

from selenium.webdriver.support.wait import WebDriverWait


import win32gui

import os
os.environ["NEW_CHAT_TOPIC"] = 'no'

# 获取当前活动的窗口
nowPageTitle = ""


# 定义一个回调函数，用于枚举所有窗口
def callback(hwnd, _):
    global nowPageTitle
    title = win32gui.GetWindowText(hwnd)
    # 打印窗口的句柄和标题
    if " - Google Chrome" in title:
#         print("title=", title)
        nowPageTitle = title.replace(" - Google Chrome", "")
#         print("nowPageTitle", nowPageTitle)


def getChromeDriver():
    ChromeDriver = ""
    try:
        ChromeDriver = ChromeDriverManager().install()
    except Exception as e:
        # print(e)
        # 提取错误信息中的版本号
        version = ""
        match = re.search(r"\d+", str(e))
        # print(match)
        if match:
            version = match.group()

        # print(version)
        if version:
            response = requests.get(
                "https://chromedriver.storage.googleapis.com/LATEST_RELEASE_" + version
            )
            # print(response.text)
            ChromeDriver = ChromeDriverManager(version=response.text).install()
            # print(ChromeDriver)

    return ChromeDriver


def getDriver(pageUrl=None, pageTitle=None, debugPort=9222):
    # 调用EnumWindows函数，传入回调函数和额外参数
    win32gui.EnumWindows(callback, None)

    chrome_options = Options()
    # chrome_options.acceptInsecureCerts = True
    # chrome_options.add_argument("--remote-debugging-port=" + str(debugPort))
    chrome_options.add_experimental_option(
        "debuggerAddress", "127.0.0.1:" + str(debugPort)
    )

    # 设置Chrome选项
    driver = webdriver.Chrome(
        service=Service(getChromeDriver()),
        options=chrome_options,
    )

    handles = driver.window_handles
    # print("handles", handles)

    title = ""

    for handle in handles:
        driver.switch_to.window(handle)
        # 启动后等待页面加载完成
        driver.implicitly_wait(1)

        # 获取当前窗口的 URL 和标题
        url = driver.current_url
        title = driver.title
        # print(handle, " == ", title, url)

        if pageUrl and (pageUrl == url or pageUrl in url):
            break
        if pageTitle and (pageTitle == title or pageTitle in title):
            break

    return driver


def getAnswer(prompt):
    driver = getDriver(pageUrl="chat.openai.com")

    if os.environ["NEW_CHAT_TOPIC"] == 'yes':
        driver.find_element(
            By.XPATH, '//*[@id="__next"]//nav/a[contains(text(),"New chat")]'
        ).click()

    # 获取输入框元素
    input_element = driver.find_element(By.XPATH, '//*[@id="prompt-textarea"]')

    # 拆分多行字符串并逐行输入
    lines = prompt.split("\n")
#     print("lines", lines)
    has = False
    for line in lines:
        input_element.send_keys(line)
        input_element.send_keys(Keys.SHIFT, Keys.ENTER)  # 模拟输入 Shift + Enter
        has = True

    time.sleep(1)
#     print("has", has)
    if has:
        input_element.send_keys(Keys.ENTER)
    else:
        input_element.send_keys(prompt + Keys.ENTER)

    time.sleep(3)
    WebDriverWait(driver, timeout=600).until(
        lambda d: d.find_element(
            By.XPATH, '//*[@id="__next"]//form/div/div[1]/div/button/div'
        ).text
        == "Regenerate response"
    )

    content = driver.find_element(
        By.XPATH,
        '//*[@id="__next"]//main//div[contains(@class,"group w-full text-gray-800 dark:text-gray-100 border-b border-black/10 dark:border-gray-900/50 bg-gray-50 dark:bg-[#444654]")][last()]//div[contains(@class,"markdown")]',
    ).text
    # .get_attribute("innerHTML")
    # print(content)

    handles = driver.window_handles
    #     print("handles2", handles)

    global nowPageTitle

    for handle in handles:
        driver.switch_to.window(handle)
        # 启动后等待页面加载完成
        driver.implicitly_wait(1)

        # 获取当前窗口的 标题
        title = driver.title

        if title == nowPageTitle:
#             print("end=====")
            break

    driver.quit()

    return content

# 对接实现LangChain的自定义LLM

class rpaForChatGPT(LLM):
    max_token: int = 10000
    temperature: float = 0.1
    top_p = 0.9
    history = []

    def __init__(self):
        super().__init__()

    @property
    def _llm_type(self) -> str:
        return "rpaForChatGPT"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response = getAnswer(prompt)
        self.history = self.history + [[None, response]]
        return response
